In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
login(HUGGINGFACE_API_KEY)

In [ ]:
import os
import glob
from datasets import load_dataset, Dataset, concatenate_datasets

dataset_id = "Savoxism/andrew-tate-long-form-raw"
data_dir = "tate_long_form_data"

new_examples = []
for txt_path in glob.glob(os.path.join(data_dir, "*.txt")):
    title = os.path.splitext(os.path.basename(txt_path))[0]
    with open(txt_path, "r", encoding="utf-8") as f:
        content = f.read().strip()
    new_examples.append({"title": title, "content": content})

if not new_examples:
    print("No .txt files found in", data_dir)

In [ ]:
from pprint import pprint
pprint(new_examples[0]['content'][:100])

In [ ]:
def push_to_hf(new_examples: list, dataset_id: str):
    try:
        ds_remote = load_dataset(dataset_id, split="train")
        print(f">>> Đã load {len(ds_remote)} examples từ HF.")
        ds_all = concatenate_datasets([ds_remote, Dataset.from_list(new_examples)])
    except Exception as e:
        print(f">>> Chưa có dataset trên HF hoặc load lỗi ({e}), sẽ tạo mới.")
        ds_all = Dataset.from_list(new_examples)
    ds_all.push_to_hub(dataset_id)
    print(f">>> Pushed tổng cộng {len(ds_all)} examples lên {dataset_id} ‹train›")

# 3. Execute the push
push_to_hf(new_examples, dataset_id)